In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/insta_data/'

import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
from tqdm.notebook import tqdm
import string
import re
from collections import defaultdict



import glob 
fileNames = glob.glob(root_path + "/*.csv")

lst = []

for fileName in fileNames:
    print(fileName)
    df = pd.read_csv(fileName, index_col=None, header=0)
    lst.append(df)

wf = pd.concat(lst, axis=0, ignore_index=True)

post_df = wf["text"]
hashtag_df = wf["hashtags"]
comments_df = wf["comments"]

#https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',str(text))

# appending first 25 comments into one big comment
def appendComments(comment):
  lst = comment.split("|")
  final_comment = ""
  for idx,entry in enumerate(lst):
    if idx == 25:
      break
    entry.strip()
    final_comment +=  entry + " "
  return final_comment

def cleanComment(comment):
  deEmojified_comment = deEmojify(comment)
  return appendComments(deEmojified_comment)


wf["comments"] = wf["comments"].apply(lambda x:cleanComment(x))
#combining comments and actual post
wf["text_with_comments"] = wf["text"]+ " "+ wf["comments"]
# print(df['hashtags'])
# print(df['text_with_comments'])
df = pd.DataFrame()
df['text_with_comments']=wf['text_with_comments']
df['hashtags']=wf['hashtags']
print(df.head())
print(df.shape)

Mounted at /content/gdrive
gdrive/My Drive/insta_data/mitpics.csv
gdrive/My Drive/insta_data/yoga.csv
gdrive/My Drive/insta_data/hidden_shots_.csv
gdrive/My Drive/insta_data/viceindia.csv
gdrive/My Drive/insta_data/healthyfoodvideos.csv
gdrive/My Drive/insta_data/pbsnature.csv
gdrive/My Drive/insta_data/eucouncil.csv
gdrive/My Drive/insta_data/afpsport.csv
gdrive/My Drive/insta_data/nature.research.csv
gdrive/My Drive/insta_data/ageofempiresgame.csv
gdrive/My Drive/insta_data/ageofempires.csv
gdrive/My Drive/insta_data/oxford_uni.csv
gdrive/My Drive/insta_data/talentmoves.csv
gdrive/My Drive/insta_data/educationaboutearth.csv
gdrive/My Drive/insta_data/nature_africa.csv
gdrive/My Drive/insta_data/abcwnn.csv
gdrive/My Drive/insta_data/aamaadmiparty.csv
gdrive/My Drive/insta_data/voxdotcom.csv
gdrive/My Drive/insta_data/life.csv
gdrive/My Drive/insta_data/world.csv
gdrive/My Drive/insta_data/nytimesopinionart.csv
gdrive/My Drive/insta_data/commonwealth_sec.csv
gdrive/My Drive/insta_data/

In [2]:
def cleanerEng(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'[^a-z0-9]+',' ',x)
  return x

def cleanerHindi(x):
  x = str(x)
  x = re.sub(r'[-.।|,?;:<>&$₹]+',' ',x)
  return x

punctuations = string.punctuation
table = punctuations.maketrans(punctuations+string.ascii_uppercase,
                               " "*len(punctuations)+string.ascii_lowercase,)

def cleanSentence(sentence):
  # sentence.strip()
  sen = sentence.translate(table)
  return sen

def addTokens(x,start=False):
  x.append('<END>')
  if start:
    x.insert(0,'<START>')
  return list(x)
  

df.iloc[:,0] = df['text_with_comments'].apply(func=cleanerEng)
df.iloc[:,0] = df['text_with_comments'].apply(func=cleanSentence)
df.iloc[:,0] = df['text_with_comments'].apply(func= lambda x : (str(x).split()))
df.iloc[:,0] = df['text_with_comments'].apply(func= addTokens,start=False)
df.iloc[:,1] = df['hashtags'].apply(func= lambda x : (str(x).split()))
df.iloc[:,1] = df['hashtags'].apply(func= addTokens,start=False)
# df.iloc[:,1] = df['hindi'].apply(func= addTokens,start=True)

articles = df['text_with_comments'].to_numpy()
labels = df['hashtags'].to_numpy()

!pip install transformers
import tensorflow as tf
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

     |████████████████████████████████| 1.3MB 5.7MB/s 
     |████████████████████████████████| 1.1MB 23.2MB/s 
     |████████████████████████████████| 890kB 35.6MB/s 
     |████████████████████████████████| 2.9MB 45.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=ce4086d2f518bd6be24ffaf5a62e2b657b8915a8d146e868ed13ae6e7486ba52
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
max_length_test = 50
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(posts,labels):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []
  for post, label in zip(posts,labels):
    if not isinstance(post, str):
      continue
    bert_input = tokenizer.encode_plus(post,
                                       truncation=True,           
                                      add_special_tokens = True, # add [CLS], [SEP]
                                      max_length = max_length_test, # max length of the text that can go to BERT
                                      padding='max_length', # add [PAD] tokens
                                      return_attention_mask = True, # add attention mask to not focus on pad token

                                      )
  
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

def convert_to_tfdataset(articles,labels,batch_size = 30):
  dataset = encode_examples(articles,labels)
  dataset = dataset.shuffle(dataset.__len__().numpy()).batch(batch_size)
  return dataset


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( articles, labels, test_size=0.1, random_state=21,shuffle = True)
train_dataset = convert_to_tfdataset(X_train,y_train)
test_dataset = convert_to_tfdataset(X_test,y_test)

In [ ]:
from transformers import BertConfig
model_name = 'bert-base-uncased'
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained(model_name,config = config1)
config1 = BertConfig.from_pretrained(model_name)
config1.num_labels = 2


In [ ]:
from transformers import TFBertModel

model = TFBertModel.from_pretrained(model_name,config = config1)
learning_rate = 2e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
bert_history = model.fit(train_dataset,epochs=5,verbose=1,validation_data=test_dataset)